# Growing classes

When implementing much of the functionality and running the research whose artifacts live in this repository, the authors found it best to document the iterations of the research and development. However, Python insists classes should be defined in one block, complicating the iterative development of its methods. We thus write here a decorator that allows for the definition of classes one method at a time, across multiple code cells.

In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120 --ignore W293,E302

In [2]:
from contextlib import contextmanager
from dask.delayed import Delayed
import dask
from functools import reduce
import inspect
from jupytest import Suite, Report, Magic, summarize_results, assert_, eq, belong_to, is_any_of
import operator as op
import re
from typing import Callable, Sequence, Optional, cast, Set, Union

In [3]:
suite = Suite()
if __name__ == "__main__":
    suite |= Report()
    suite |= Magic()

In [4]:
Decorator = Callable[[Callable], Callable]


def growing(klass: type) -> type:
    def add_method(
        fn_: Optional[Callable] = None,
        name: str = "",
        wrapped_in: Union[Decorator, Sequence[Decorator]] = []
    ) -> Callable:
        def add_to_class(fn: Callable):
            name_method = name or fn.__name__
            method_new = reduce(lambda f, w: w(f), wrapped_in if hasattr(wrapped_in, "__iter__") else [wrapped_in], fn)
            setattr(klass, name_method, method_new)
            return getattr(klass, name_method)
        
        if fn_ is None:
            return add_to_class
        return add_to_class(cast(Callable, fn_))
    
    def add_class_method(
        fn_: Optional[Callable] = None,
        name: str = "",
        wrapped_in: Union[Decorator, Sequence[Decorator]] = []
    ) -> Callable:
        wrappers = wrapped_in if hasattr(wrapped_in, "__iter__") else [wrapped_in]
        return add_method(fn_, name, wrappers + [classmethod])

    setattr(klass, "method", staticmethod(add_method))
    setattr(klass, "classmethod", staticmethod(add_class_method))
    return klass

## Tests

In [5]:
def user_members(klass) -> Set[str]:
    return {m for m in dir(klass) if not re.match(r"^__.*__$", m)}

In [6]:
dir(op)

['__abs__',
 '__add__',
 '__all__',
 '__and__',
 '__builtins__',
 '__cached__',
 '__concat__',
 '__contains__',
 '__delitem__',
 '__doc__',
 '__eq__',
 '__file__',
 '__floordiv__',
 '__ge__',
 '__getitem__',
 '__gt__',
 '__iadd__',
 '__iand__',
 '__iconcat__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__inv__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__loader__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__name__',
 '__ne__',
 '__neg__',
 '__not__',
 '__or__',
 '__package__',
 '__pos__',
 '__pow__',
 '__rshift__',
 '__setitem__',
 '__spec__',
 '__sub__',
 '__truediv__',
 '__xor__',
 '_abs',
 'abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'inv

In [7]:
%%test Add method
@growing
class MyClass:
    def f(self):
        return 5

assert_(op.le, {"f", "method"}, user_members(MyClass), msg="User members before adding method g")
assert_(lambda x, S: x not in S, "g", user_members(MyClass), msg="Method g should not be among user members")


@MyClass.method
def g(self, x):
    return self.f() + x

assert_(op.le, {"f", "g", "method"}, user_members(MyClass), msg="User members after adding method g")
assert_(eq, obtained=MyClass().g(3), expected=8)

Test Add method passed.


In [8]:
%%test Add Dask Delayed method
@growing
class MyClass:
    def f(self):
        return 5

@MyClass.method(wrapped_in=dask.delayed(pure=True))
def h(self, x, y):
    return self.f() * x + y

assert_(belong_to(user_members(MyClass)), "h")
assert_(is_any_of(Delayed), MyClass().h(4, 5))
assert_(eq, expected=25, obtained=MyClass().h(4, 5).compute(scheduler="single-threaded"))

Test Add Dask Delayed method passed.


In [9]:
%%test Multiple method wrappers
@growing
class MyClass:
    def f(self):
        return 5

def wrapper1(fn):
    return lambda self, x: fn(self, x) + x

def wrapper2(fn):
    return lambda self, x: fn(self, x) * x

@MyClass.method(wrapped_in=[wrapper1, wrapper2])
def double_wrapped(self, x):
    return x / 3 + self.f()

assert_(belong_to(user_members(MyClass)), "double_wrapped")
assert_(eq, expected=153.0, obtained=MyClass().double_wrapped(9))

Test Multiple method wrappers passed.


In [10]:
%%test Add class method, inelegant
@growing
class MyClass:
    C = 34
    
    def f(self):
        return 5

try:
    @MyClass.method
    @classmethod
    def cm(cls):
        return cls.C
    fail()
except AttributeError:
    pass


@MyClass.method(wrapped_in=classmethod)
def cm(cls):
    return cls.C


assert_(eq, expected=MyClass.C, obtained=MyClass.cm())

Test Add class method, inelegant passed.


In [11]:
%%test Add class method, preferred approach
@growing
class MyClass:
    C = 34
    
    def f(self):
        return 5

@MyClass.classmethod
def cm(cls):
    return cls.C


assert_(eq, expected=MyClass.C, obtained=MyClass.cm())

Test Add class method, preferred approach passed.


In [12]:
%%test Add class method that acts as context manager
@growing
class MyClass:
    C = 34
    def f(self):
        return 5
    
    
@MyClass.classmethod(wrapped_in=contextmanager)
def changing_C(cls, num: int):
    old = cls.C
    try:
        cls.C = num
        yield
    finally:
        cls.C = old
        
        
assert_(eq, expected=34, obtained=MyClass.C)
with MyClass.changing_C(45):
    assert_(eq, expected=45, obtained=MyClass.C)
assert_(eq, expected=34, obtained=MyClass.C)

Test Add class method that acts as context manager passed.


In [13]:
%%test Add method, then redefine it
@growing
class C:
    def f(self):
        return 56
    

assert_(eq, expected=56, obtained=C().f())


@C.method
def f(self):
    return 890


assert_(eq, expected=890, obtained=C().f())

Test Add method, then redefine it passed.


In [14]:
if __name__ == "__main__":
    _ = summarize_results(suite)

7 passed, 0 failed, 0 raised an error
